## 모듈 설정

In [1]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertModel, AdamW
from torch.nn import CrossEntropyLoss
device = torch.device("cuda")

/home/kpkim/.pyenv/versions/3.7.7/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## 다운받은 json 파일의 데이터 불러오기

In [2]:
import json
from pathlib import Path

# path_train = Path('KorQuAD/train-v1.0.json')
# with open(path_train, "r", encoding='utf-8') as reader:
#     json_train = json.load(reader)["data"]
    
path_dev = Path('KorQuAD/dev-v1.0.json')
with open(path_dev, "r", encoding='utf-8') as reader:
    json_dev = json.load(reader)["data"]

In [3]:
# parsing된 데이터의 타입 확인

# print(type(json_train))
print(type(json_dev))

<class 'list'>


## json parsing (read_squad)

### function

In [4]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answer_texts = []
    answer_start_positions = []
    answer_end_positions = []
    length_of_answer = 0
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    answer_text = answer["text"]
                    length_of_answer = len(answer_text)
                    start_number = answer["answer_start"]
                    end_number = start_number + length_of_answer
                    
                    contexts.append(context)
                    questions.append(question)
                    answer_texts.append(answer_text)
                    answer_start_positions.append(start_number)
                    answer_end_positions.append(end_number)
                    

    return contexts, questions, answer_texts, answer_start_positions, answer_end_positions

### data 함수 처리

In [5]:
# train_context, train_que, train_ans, train_start_pos, train_end_pos, train_words_maps = read_squad('KorQuAD/train-v1.0.json')

val_context, val_que, val_ans, val_start_pos, val_end_pos = read_squad('KorQuAD/dev-v1.0.json')

In [6]:
# val_context : 지문
# val_que : 질문
# val_ans : 정답
# val_start_pos : 지문에서 정답의 시작 index
# val_end_pos : 지문에서 정답의 끝 index

### data type 확인

In [7]:
print(type(val_context))
print(type(val_que))
print(type(val_ans))
print(type(val_start_pos))
print(type(val_end_pos))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


### data length 확인

In [8]:
print(len(val_context))
print(len(val_que))
print(len(val_ans))
print(len(val_start_pos))
print(len(val_end_pos))

5774
5774
5774
5774
5774


### sample data 확인

#### train

In [9]:
# sample_number = 50
# print(train_context[sample_number], '\n')
# print('질문 : ', train_que[sample_number], '\n')
# print('본문 길이:', len(train_context[sample_number]), '\n')
# print('answer_start: ', train_start_pos[sample_number])
# print('answer_end: ', train_end_pos[sample_number], '\n')
# print('정답 (train_ans) : ', train_ans[sample_number])
# print('정답 (train_pos) : ', train_context[sample_number][train_start_pos[sample_number]:train_end_pos[sample_number]])

#### validation (sample)

In [70]:
## print로 데이터 형태를 확인할 sample의 번호를 지정 (sample_number / 5774)

sample_number = 2000

In [10]:
print(val_context[sample_number], '\n')
print('질문 : ', val_que[sample_number], '\n')
print('본문 길이:', len(val_context[sample_number]), '\n')
print('answer_start: ', val_start_pos[sample_number])
print('answer_end: ', val_end_pos[sample_number], '\n')
print('정답 (val_ans) : ', val_ans[sample_number])
print('정답 (val_pos) : ', val_context[sample_number][val_start_pos[sample_number]:val_end_pos[sample_number]])

그 중에서도 여자 스키점프를 정식 종목으로 추가하는 것이 가장 논란이 되었다. 여자 스키점프에 대한 논란은 2009년 4월 21일 ~ 24일 사이에 밴쿠버에 위치한 브리티시 컬럼비아 주 대법원에서 열린 공판에서 불허 판결이 나오고 그 해 7월 10일, 2010년 동계 올림픽 이후로도 정식 종목으로 포함될 수 없다는 결정이 나온 후 더욱 커졌다. 이에 따라 여자 스키 점프의 올림픽 정식 종목 채택에 대해 찬성하는 사람들은 러시아의 소치에서 열리는 2014년 동계 올림픽에서 정식 종목으로 추가될 수 있도록 노력하고 있다. 이 판결에 대한 불만을 완화하기위해 VANOC에서는 캐나다 전국의 여성들을 휘슬러 올림픽 공원에서 열리는(2009년 1월에 열리는 대륙간 컵 포함)것에 초대했다. 

질문 :  2009년 밴쿠버에 위치한 브리티시 컬럼비아 주 대법원에서 열린 공판에서 불허 판결이 나오고 2010년 동계 올림픽 이후로도 정식 종목으로 포함될 수 없다는 결정이 난 동계 올림픽 종목은? 

본문 길이: 383 

answer_start:  7
answer_end:  14 

정답 (val_ans) :  여자 스키점프
정답 (val_pos) :  여자 스키점프


#### validation (multi sample)

In [18]:
def print_answer_position(number):
    print('answer_text: ', val_ans[number])
    print('answer_start: ', val_start_pos[number], 'answer_end: ', val_end_pos[number], '\n')

In [19]:
for i in range(2000, 2020):
    print_answer_position(i)

answer_text:  여자 스키점프
answer_start:  7 answer_end:  14 

answer_text:  소치
answer_start:  242 answer_end:  244 

answer_text:  VANOC
answer_start:  313 answer_end:  318 

answer_text:  여자 스키점프
answer_start:  7 answer_end:  14 

answer_text:  밴쿠버
answer_start:  83 answer_end:  86 

answer_text:  SBS
answer_start:  21 answer_end:  24 

answer_text:  IOC와의 올림픽 중계 독점계약 절차
answer_start:  191 answer_end:  211 

answer_text:  일본
answer_start:  648 answer_end:  650 

answer_text:  SBS
answer_start:  21 answer_end:  24 

answer_text:  SBS
answer_start:  566 answer_end:  569 

answer_text:  리우데자네이루
answer_start:  481 answer_end:  488 

answer_text:  SBS
answer_start:  21 answer_end:  24 

answer_text:  대한민국
answer_start:  596 answer_end:  600 

answer_text:  텔만
answer_start:  316 answer_end:  318 

answer_text:  2명
answer_start:  21 answer_end:  23 

answer_text:  2명
answer_start:  21 answer_end:  23 

answer_text:  공산당
answer_start:  272 answer_end:  275 

answer_text:  파울 폰 힌덴부르크
answer_start: 

## splitted words from context

### function

In [20]:
def generate_original_words(context_data):
    context_words_arr = []
    for context in context_data:
        split_words = context.split(' ')
        context_words_arr.append(split_words)
    return context_words_arr

### data 함수 처리

In [21]:
# train_split_words = generate_original_words(train_context)

val_split_words = generate_original_words(val_context)

In [22]:
# val_split_words : 지문을 단어 별로 나눠서 저장한 list

### sample data 확인

#### train

In [15]:
# print(train_split_words[sample_number])
# print(len(train_split_words[sample_number]))

#### validation (sample)

In [23]:
print(val_split_words[sample_number], '\n')
print('number of words: ', len(val_split_words[sample_number]))

['그', '중에서도', '여자', '스키점프를', '정식', '종목으로', '추가하는', '것이', '가장', '논란이', '되었다.', '여자', '스키점프에', '대한', '논란은', '2009년', '4월', '21일', '~', '24일', '사이에', '밴쿠버에', '위치한', '브리티시', '컬럼비아', '주', '대법원에서', '열린', '공판에서', '불허', '판결이', '나오고', '그', '해', '7월', '10일,', '2010년', '동계', '올림픽', '이후로도', '정식', '종목으로', '포함될', '수', '없다는', '결정이', '나온', '후', '더욱', '커졌다.', '이에', '따라', '여자', '스키', '점프의', '올림픽', '정식', '종목', '채택에', '대해', '찬성하는', '사람들은', '러시아의', '소치에서', '열리는', '2014년', '동계', '올림픽에서', '정식', '종목으로', '추가될', '수', '있도록', '노력하고', '있다.', '이', '판결에', '대한', '불만을', '완화하기위해', 'VANOC에서는', '캐나다', '전국의', '여성들을', '휘슬러', '올림픽', '공원에서', '열리는(2009년', '1월에', '열리는', '대륙간', '컵', '포함)것에', '초대했다.'] 

number of words:  94


#### validation (multi sample)

In [26]:
def print_splits(number):
    print(val_split_words[number])
    print('number of words: ', len(val_split_words[number]), '\n')

In [27]:
for i in range(2000, 2020):
    print_splits(i)

['그', '중에서도', '여자', '스키점프를', '정식', '종목으로', '추가하는', '것이', '가장', '논란이', '되었다.', '여자', '스키점프에', '대한', '논란은', '2009년', '4월', '21일', '~', '24일', '사이에', '밴쿠버에', '위치한', '브리티시', '컬럼비아', '주', '대법원에서', '열린', '공판에서', '불허', '판결이', '나오고', '그', '해', '7월', '10일,', '2010년', '동계', '올림픽', '이후로도', '정식', '종목으로', '포함될', '수', '없다는', '결정이', '나온', '후', '더욱', '커졌다.', '이에', '따라', '여자', '스키', '점프의', '올림픽', '정식', '종목', '채택에', '대해', '찬성하는', '사람들은', '러시아의', '소치에서', '열리는', '2014년', '동계', '올림픽에서', '정식', '종목으로', '추가될', '수', '있도록', '노력하고', '있다.', '이', '판결에', '대한', '불만을', '완화하기위해', 'VANOC에서는', '캐나다', '전국의', '여성들을', '휘슬러', '올림픽', '공원에서', '열리는(2009년', '1월에', '열리는', '대륙간', '컵', '포함)것에', '초대했다.']
number of words:  94 

['그', '중에서도', '여자', '스키점프를', '정식', '종목으로', '추가하는', '것이', '가장', '논란이', '되었다.', '여자', '스키점프에', '대한', '논란은', '2009년', '4월', '21일', '~', '24일', '사이에', '밴쿠버에', '위치한', '브리티시', '컬럼비아', '주', '대법원에서', '열린', '공판에서', '불허', '판결이', '나오고', '그', '해', '7월', '10일,', '2010년', '동계', '올림픽', '이후로도', '정식', '종목으로', '포함될', '수', '없다는

## get_space_arr - not necessary

### function

In [28]:
def get_space_arr(context):
    arr = []
    for i in range(len(context)):
        if context[i] == ' ':
            arr.append(i+1)
    arr.append(len(context))
    return arr

### validation data 함수 처리

In [29]:
# train_space_arr = []
# for context in train_context:
#     train_space_arr.append(get_space_arr(context))

In [30]:
val_space_arr = []
for context in val_context:
    val_space_arr.append(get_space_arr(context))

### sample data 확인

#### train

In [31]:
# print(len(train_space_arr))
# print(train_space_arr[sample_number])

#### validation (sample)

In [32]:
print(len(val_space_arr), '\n')
print(val_space_arr[sample_number])

5774 

[2, 7, 10, 16, 19, 24, 29, 32, 35, 39, 44, 47, 53, 56, 60, 66, 69, 73, 75, 79, 83, 88, 92, 97, 102, 104, 110, 113, 118, 121, 125, 129, 131, 133, 136, 141, 147, 150, 154, 159, 162, 167, 171, 173, 177, 181, 184, 186, 189, 194, 197, 200, 203, 206, 210, 214, 217, 220, 224, 227, 232, 237, 242, 247, 251, 257, 260, 266, 269, 274, 278, 280, 284, 289, 293, 295, 299, 302, 306, 313, 322, 326, 330, 335, 339, 343, 348, 358, 362, 366, 370, 372, 378, 383]


## word_start_position, word_end_position, char_to_word_offset

### function (start_position, end_position, char_to_word_offset)

In [35]:
def _is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

def word_start_end_offset(context, ans, start_pos):

    doc_tokens = []
    char_to_word_offset = []
    prev_is_whitespace = True
    answer_offset = start_pos

    for c in context:
        if _is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                doc_tokens.append(c)
            else:
                doc_tokens[-1] += c
            prev_is_whitespace = False
        char_to_word_offset.append(len(doc_tokens) - 1)

    word_start_position = char_to_word_offset[answer_offset]
    word_end_position = char_to_word_offset[answer_offset + len(ans) - 1]
    return word_start_position, word_end_position, char_to_word_offset

### data 함수 처리

In [36]:
val_word_start = []
val_word_end = []
val_char_word_offset = []
for i in range(len(val_context)):
    word_start, word_end, char_word_offset = word_start_end_offset(val_context[i], val_ans[i], val_start_pos[i])
    val_word_start.append(word_start)
    val_word_end.append(word_end)
    val_char_word_offset.append(char_word_offset)

In [37]:
# val_word_start : 단어 list에서 정답의 시작 위치 index
# val_word_end : 단어 list에서 정답의 끝 위치 index
# val_char_word_offset : 각 토큰이 몇 번째 (단어+space)의 위치에 대응되는지 기록한 list

### sample data 확인

#### train

In [26]:
# print(train_word_start[sample_number], train_word_end[sample_number])
# print(' '.join(train_split_words[sample_number][train_word_start[sample_number]:train_word_end[sample_number]+1]))
# print(train_char_word_offset[sample_number])

In [27]:
# print(train_char_word_offset[sample_number].index(train_word_start[sample_number]))
# print(train_start_pos[sample_number], '\n')
# print(train_char_word_offset[sample_number].index(train_word_end[sample_number]+1)-1)
# print(train_end_pos[sample_number])

#### validation (sample)

In [38]:
print(val_context[sample_number], '\n')
print(val_char_word_offset[sample_number], '\n')
print('answer word start position : ', val_word_start[sample_number])
print('answer word end position : ', val_word_end[sample_number], '\n')
print(' '.join(val_split_words[sample_number][val_word_start[sample_number]:val_word_end[sample_number]+1]), '\n')
print(len(val_char_word_offset[sample_number]))
print(len(val_context[sample_number]))

그 중에서도 여자 스키점프를 정식 종목으로 추가하는 것이 가장 논란이 되었다. 여자 스키점프에 대한 논란은 2009년 4월 21일 ~ 24일 사이에 밴쿠버에 위치한 브리티시 컬럼비아 주 대법원에서 열린 공판에서 불허 판결이 나오고 그 해 7월 10일, 2010년 동계 올림픽 이후로도 정식 종목으로 포함될 수 없다는 결정이 나온 후 더욱 커졌다. 이에 따라 여자 스키 점프의 올림픽 정식 종목 채택에 대해 찬성하는 사람들은 러시아의 소치에서 열리는 2014년 동계 올림픽에서 정식 종목으로 추가될 수 있도록 노력하고 있다. 이 판결에 대한 불만을 완화하기위해 VANOC에서는 캐나다 전국의 여성들을 휘슬러 올림픽 공원에서 열리는(2009년 1월에 열리는 대륙간 컵 포함)것에 초대했다. 

[0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 17, 17, 17, 17, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 25, 25, 26, 26, 26, 26, 26, 26, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 33, 33, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 37, 37, 37, 38, 38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 41, 

In [71]:
print('start_pos : ',val_start_pos[sample_number], '/ end_pos : ', val_end_pos[sample_number], '\n')
print(
    'char_word_offset으로 찾은 answer 위치 : ',
    val_char_word_offset[sample_number].index(val_word_start[sample_number]),'/',
    val_char_word_offset[sample_number].index(val_word_end[sample_number]+1)-1
)

start_pos :  7 / end_pos :  14 

char_word_offset으로 찾은 answer 위치 :  7 / 15


#### validation (multi sample)

In [46]:
def print_start_end_offset(number):
    print('word start : ', val_word_start[number], '/', 'word end : ', val_word_end[number])
    print('answer words :',' '.join(val_split_words[number][val_word_start[number]:val_word_end[number]+1]))
    print('answer text : ', val_ans[number], '\n')

In [47]:
for i in range(2000, 2020):
    print_start_end_offset(i)

word start :  2 / word end :  3
answer words : 여자 스키점프를
answer text :  여자 스키점프 

word start :  63 / word end :  63
answer words : 소치에서
answer text :  소치 

word start :  80 / word end :  80
answer words : VANOC에서는
answer text :  VANOC 

word start :  2 / word end :  3
answer words : 여자 스키점프를
answer text :  여자 스키점프 

word start :  21 / word end :  21
answer words : 밴쿠버에
answer text :  밴쿠버 

word start :  5 / word end :  5
answer words : SBS에서
answer text :  SBS 

word start :  42 / word end :  46
answer words : IOC와의 올림픽 중계 독점계약 절차에
answer text :  IOC와의 올림픽 중계 독점계약 절차 

word start :  141 / word end :  141
answer words : 일본(JPN)
answer text :  일본 

word start :  5 / word end :  5
answer words : SBS에서
answer text :  SBS 

word start :  123 / word end :  123
answer words : SBS는
answer text :  SBS 

word start :  104 / word end :  104
answer words : 리우데자네이루에서
answer text :  리우데자네이루 

word start :  5 / word end :  5
answer words : SBS에서
answer text :  SBS 

word start :  131 / word end :  131

## Tokenizer & Tokenizing

### pre_trained tokenizer 세팅

In [48]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

### context data - tokenize

In [50]:
# train_encodings = tokenizer(train_contexts, train_ques, max_length = 512, truncation=True, padding=True)

val_encodings = tokenizer(val_context, max_length=512, truncation=True, padding='max_length')

### sample data 확인

#### train

In [33]:
# print(train_encodings.keys(), '\n')
# print(len(sample_id), '\n')
# print(sample_id)

#### validation (sample)

In [56]:
sample_ids = val_encodings.data['input_ids'][sample_number]

print(val_encodings.keys(), '\n')
print(len(sample_ids), '\n')
print(sample_ids)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask']) 

512 

[101, 8924, 102246, 12424, 12092, 62592, 9477, 21039, 34907, 28396, 11513, 9670, 21155, 9684, 68055, 11467, 9765, 11287, 12178, 27487, 22224, 9024, 49919, 10739, 17737, 119, 62592, 9477, 21039, 34907, 28396, 10530, 18154, 9024, 49919, 10892, 20067, 16464, 45966, 198, 47251, 64932, 9332, 61156, 41605, 10530, 39828, 9376, 12692, 45725, 14040, 9801, 118866, 72803, 9689, 9069, 33768, 108280, 12424, 66421, 8896, 33323, 11489, 9368, 119425, 9903, 74322, 10739, 8982, 28188, 11664, 8924, 9960, 16743, 39335, 117, 19145, 9095, 21611, 85735, 18347, 11261, 12092, 9670, 21155, 9684, 68055, 11467, 9928, 48533, 59330, 9460, 39218, 11018, 8881, 98489, 8982, 37093, 10003, 99958, 9798, 32855, 119, 41099, 22799, 62592, 9477, 21039, 9668, 28396, 10459, 85735, 9670, 21155, 9684, 68055, 9738, 119342, 10530, 33378, 9732, 17138, 12178, 9405, 61250, 22879, 62834, 10459, 9448, 18622, 11489, 9569, 26344, 18670, 9095, 21611, 85735, 11489, 9670, 

## context를 tokenize - not necessary

### validation encodings data - tokenize

In [35]:
# train_tokens = []
# for i in range(len(train_encodings.data['input_ids'])):
#     train_tokens.append(tokenizer.convert_ids_to_tokens(train_encodings.data['input_ids'][i]))

In [36]:
val_tokens = []
for i in range(len(val_encodings.data['input_ids'])):
    val_tokens.append(tokenizer.convert_ids_to_tokens(val_encodings.data['input_ids'][i]))

### sample data 확인

In [37]:
# print(len(train_encodings.data['input_ids']))

print(len(val_encodings.data['input_ids']))

5774


In [38]:
# print(train_tokens[sample_number])

In [39]:
print(val_tokens[sample_number])

['[CLS]', '그', '중에', '##서', '##도', '여자', '스', '##키', '##점', '##프', '##를', '정', '##식', '종', '##목', '##으로', '추', '##가', '##하는', '것이', '가장', '논', '##란', '##이', '되었다', '.', '여자', '스', '##키', '##점', '##프', '##에', '대한', '논', '##란', '##은', '2009년', '4월', '21일', '~', '24일', '사이에', '밴', '##쿠', '##버', '##에', '위치한', '브', '##리', '##티', '##시', '컬', '##럼', '##비아', '주', '대', '##법', '##원에', '##서', '열린', '공', '##판', '##에서', '불', '##허', '판', '##결', '##이', '나', '##오', '##고', '그', '해', '7월', '10일', ',', '2010년', '동', '##계', '올림픽', '이후', '##로', '##도', '정', '##식', '종', '##목', '##으로', '포', '##함', '##될', '수', '없다', '##는', '결', '##정이', '나', '##온', '후', '더욱', '커', '##졌다', '.', '이에', '따라', '여자', '스', '##키', '점', '##프', '##의', '올림픽', '정', '##식', '종', '##목', '채', '##택', '##에', '대해', '찬', '##성', '##하는', '사', '##람', '##들은', '러시아', '##의', '소', '##치', '##에서', '열', '##리는', '2014년', '동', '##계', '올림픽', '##에서', '정', '##식', '종', '##목', '##으로', '추', '##가', '##될', '수', '있도록', '노', '##력', '##하고', '있다', '.', '이', '판', '##결', '

## split_words를 tokenize

### function

In [53]:
def words_to_tokens(split_words):
    sub_to_tok_index = []
    all_doc_tokens = []
    word_token_number = []
    for idx, word in enumerate(split_words):
        sub_tokens = tokenizer.tokenize(word)
        j = 0
        for sub_token in sub_tokens:
            sub_to_tok_index.append(idx)
            all_doc_tokens.append(sub_token)
            j += 1
        word_token_number.append(j)
    return sub_to_tok_index, all_doc_tokens, word_token_number

### data 함수 처리

In [54]:
val_sub_tok_index = []
val_all_tokens = []
val_word_token_number = []
for split_words in val_split_words:
    sub_tok_index, all_tokens, word_token  = words_to_tokens(split_words)
    val_sub_tok_index.append(sub_tok_index)
    val_all_tokens.append(all_tokens)
    val_word_token_number.append(word_token)

In [57]:
# val_sub_tok_index : context의 모든 subtoken이 몇 번째 위치의 단어에 대응되는지 기록한 list
# val_all_tokens : context를 단어 단위로 자른 후 단어들을 다시 tokenizing하여 모든 토큰을 저장
# val_word_token_number : context에서 각 단어가 몇 개의 토큰인지 기록한 list

### sample data 확인

#### train

In [43]:
# print(len(train_sub_tok_index))
# print(len(train_all_tokens))
# print(train_sub_tok_index[sample_number])

#### validation (sample)

In [77]:
print('sub_tok_index 길이 : ',len(val_sub_tok_index[sample_number]), 
      '/ all_tokens 길이 : ', len(val_all_tokens[sample_number]), '\n')
print('각 단어의 토큰 개수 list : ','\n', val_word_token_number[sample_number], '\n')
print('각 토큰의 대응되는 단어 순번 list : ','\n',val_sub_tok_index[sample_number],'\n')
print(val_all_tokens[sample_number], '\n')
print('word_token_number 길이 : ', len(val_word_token_number[sample_number]),
      '/ split_words 길이 : ',len(val_split_words[sample_number]))

sub_tok_index 길이 :  201 / all_tokens 길이 :  201 

각 단어의 토큰 개수 list :  
 [1, 3, 1, 5, 2, 3, 3, 1, 1, 3, 2, 1, 5, 1, 3, 1, 1, 1, 1, 1, 1, 4, 1, 4, 3, 1, 4, 1, 3, 2, 3, 3, 1, 1, 1, 2, 1, 2, 1, 3, 2, 3, 3, 1, 2, 2, 2, 1, 1, 3, 1, 1, 1, 2, 3, 1, 2, 2, 3, 1, 3, 3, 2, 3, 2, 1, 2, 2, 2, 3, 3, 1, 1, 3, 2, 1, 3, 1, 3, 5, 4, 2, 2, 2, 3, 1, 3, 4, 2, 2, 3, 1, 4, 3] 

각 토큰의 대응되는 단어 순번 list :  
 [0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 8, 9, 9, 9, 10, 10, 11, 12, 12, 12, 12, 12, 13, 14, 14, 14, 15, 16, 17, 18, 19, 20, 21, 21, 21, 21, 22, 23, 23, 23, 23, 24, 24, 24, 25, 26, 26, 26, 26, 27, 28, 28, 28, 29, 29, 30, 30, 30, 31, 31, 31, 32, 33, 34, 35, 35, 36, 37, 37, 38, 39, 39, 39, 40, 40, 41, 41, 41, 42, 42, 42, 43, 44, 44, 45, 45, 46, 46, 47, 48, 49, 49, 49, 50, 51, 52, 53, 53, 54, 54, 54, 55, 56, 56, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 61, 61, 62, 62, 63, 63, 63, 64, 64, 65, 66, 66, 67, 67, 68, 68, 69, 69, 69, 70, 70, 70, 71, 72, 73, 73, 73, 74, 74, 75, 76, 76, 76, 77, 78, 78, 78,

## ans_start_token_num, ans_end_token_num

### function

In [78]:
def ans_token_position(sub_tok_index, word_start, word_end, word_token_number):
    ans_start_token_number = sub_tok_index.index(word_start)
    ans_end_token_number = sub_tok_index.index(word_end) + word_token_number[word_end]
    return ans_start_token_number, ans_end_token_number

### data 함수 처리

In [79]:
val_ans_start_token_number = []
val_ans_end_token_number = []
for i in range(len(val_context)):
    start_num, end_num = ans_token_position(val_sub_tok_index[i], val_word_start[i], val_word_end[i], val_word_token_number[i])
    val_ans_start_token_number.append(start_num)
    val_ans_end_token_number.append(end_num)

In [82]:
# val_ans_start_token_number : answer가 들어있는 첫 단어의 첫 토큰 위치
# val_ans_end_token_number : answer가 들어있는 마지막 하나 전 단어의 위치 + 마지막 단어의 token 개수

### sample data 확인

#### train

In [80]:
# start = train_ans_start_token_number[sample_number]
# end = train_ans_end_token_number[sample_number]
# print(start, end)
# print(train_all_tokens[sample_number][start],'...', train_all_tokens[sample_number][end])
# print(train_all_tokens[sample_number][start:end + 1])
# print(train_ans[sample_number])

#### validation (sample)

In [86]:
start = val_ans_start_token_number[sample_number]
end = val_ans_end_token_number[sample_number]
print(start, end, '\n')
print('first/last token : ',val_all_tokens[sample_number][start],' . . . ', val_all_tokens[sample_number][end - 2], '\n')
print('answer tokens : ',val_all_tokens[sample_number][start:end - 1])
print('answer text : ',val_ans[sample_number])

4 10 

first/last token :  여자  . . .  ##프 

answer tokens :  ['여자', '스', '##키', '##점', '##프']
answer text :  여자 스키점프


#### validation (multi sample)

In [96]:
def print_answer_token_position(number):
    start = val_ans_start_token_number[number]
    end = val_ans_end_token_number[number]
    print('first/last token : ',val_all_tokens[number][start],' . . . ', val_all_tokens[number][end-2])
    print('answer tokens : ', val_all_tokens[number][start:end - 1])
    print('answer text : ', val_ans[number], '\n')

In [98]:
for i in range(2000, 2020):
    print_answer_token_position(i)

first/last token :  여자  . . .  ##프
answer tokens :  ['여자', '스', '##키', '##점', '##프']
answer text :  여자 스키점프 

first/last token :  소  . . .  ##치
answer tokens :  ['소', '##치']
answer text :  소치 

first/last token :  VA  . . .  ##C
answer tokens :  ['VA', '##NO', '##C']
answer text :  VANOC 

first/last token :  여자  . . .  ##프
answer tokens :  ['여자', '스', '##키', '##점', '##프']
answer text :  여자 스키점프 

first/last token :  밴  . . .  ##버
answer tokens :  ['밴', '##쿠', '##버']
answer text :  밴쿠버 

first/last token :  SBS  . . .  SBS
answer tokens :  ['SBS']
answer text :  SBS 

first/last token :  IOC  . . .  ##차
answer tokens :  ['IOC', '##와의', '올림픽', '중', '##계', '독', '##점', '##계', '##약', '절', '##차']
answer text :  IOC와의 올림픽 중계 독점계약 절차 

first/last token :  일본  . . .  ##N
answer tokens :  ['일본', '(', 'JP', '##N']
answer text :  일본 

first/last token :  SBS  . . .  SBS
answer tokens :  ['SBS']
answer text :  SBS 

first/last token :  SBS  . . .  SBS
answer tokens :  ['SBS']
answer text :  SBS 



## encoding에 start_positions, end_positions 추가

### function

In [99]:
def add_token_positions(encodings, ans_start_token_number, ans_end_token_number):
    start_positions = []
    end_positions = []
    for i in range(len(encodings.data['input_ids'])):
        start_positions.append(ans_start_token_number[i])
        end_positions.append(ans_end_token_number[i])
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

### validation data 함수 처리

In [100]:
# add_token_positions(train_encodings, train_ans_start_token_number, train_ans_end_token_number)

add_token_positions(val_encodings, val_ans_start_token_number, val_ans_end_token_number)

### sample data 확인

In [108]:
# print(train_encodings.keys())

print(val_encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


#### train

In [55]:
# start = train_encodings.data['start_positions'][sample_number]
# end = train_encodings.data['end_positions'][sample_number]
# print(start, end)
# print(train_all_tokens[sample_number][start],'...', train_all_tokens[sample_number][end])
# print(train_all_tokens[sample_number][start:end + 1])
# print(train_ans[sample_number])

#### validation (sample)

In [102]:
start = val_encodings.data['start_positions'][sample_number]
end = val_encodings.data['end_positions'][sample_number]
print(start, end, '\n')
print(val_all_tokens[sample_number][start],'...', val_all_tokens[sample_number][end - 1], '\n')
print('answer tokens : ', val_all_tokens[sample_number][start:end - 1])
print('answer text : ', val_ans[sample_number])

4 10 

여자 ... ##를 

['여자', '스', '##키', '##점', '##프']
여자 스키점프


#### validation (all data)

In [128]:
def answer_start_end_encodings(number):
    start = val_encodings.data['start_positions'][number]
    end = val_encodings.data['end_positions'][number]
    print(val_all_tokens[number][start:end], '  /  ', val_ans[number])

In [129]:
for i in range(len(val_encodings.data['start_positions'])):
    answer_start_end_encodings(i)

['1989년', '2월', '15일']   /   1989년 2월 15일
['임', '##수', '##경', '##을']   /   임수경
['1989년']   /   1989년
['학', '##생', '##회', '##관', '건', '##물', '계', '##단', '##을']   /   학생회관 건물 계단
['서울', '##지', '##방', '##경', '##찰', '##청', '공', '##안', '##분', '##실', '##로']   /   서울지방경찰청 공안분실
['임', '##종', '##석', '##의']   /   임종석
['여', '##의', '##도', '농', '##민', '폭', '##력', '시', '##위를']   /   여의도 농민 폭력 시위
['허', '##영']   /   허영
['10', '##차', '개', '##헌', '##안', '발', '##표', '##이다', '.']   /   10차 개헌안 발표
['한다', "'", '(', '제', '##8', '##9', '##조', ')', '는']   /   제89조
['허', '##영']   /   허영
['미국', '육', '##군', '부', '##참', '##모', '총', '##장', '##과']   /   미국 육군 부참모 총장
['초대', '국', '##무', '##장', '##관', '##직', '##을']   /   초대 국무장관직
['로', '##널', '##드', '레', '##이', '##건', '대통령']   /   로널드 레이건 대통령
['알', '##렉', '##산', '##더', '메', '##이', '##그', '##스', '헤', '##이', '##그', '2', '##세', '(', '영어', ':']   /   알렉산더 메이그스 헤이그 2세
['미국', '육', '##군', '부', '##참', '##모', '총', '##장', '##과']   /   미국 육군 부참모 총장
['1924', '##년', '12월', '2일']   / 

["'", '지', '##식', "'", '이라는']   /   지식
['임', '##호', '##테', '##프', '##는']   /   임호테프
['아', '##이', '##작', '아', '##시', '##모', '##프', '##는']   /   아이작 아시모프
['임', '##호', '##테', '##프', '##는']   /   임호테프
['알', '##렉', '##산', '##드', '##리아', '도', '##서', '##관은']   /   알렉산드리아 도서관
['러시아']   /   러시아
['아', '##이', '##작', '아', '##시', '##모', '##프', '##는']   /   아이작 아시모프
['임', '##호', '##테', '##프', '##는']   /   임호테프
['탈', '##동', '##조', '##화가']   /   탈동조화
['감', '##마', '##선', '망', '##원', '##경']   /   감마선 망원경
['콜', '##모', '##고', '##로', '##프', '-', '스', '##미', '##르', '##노', '##프', '검', '##정을']   /   콜모고로프-스미르노프 검정
['탈', '##동', '##조', '##화가']   /   탈동조화
['"', '거', '##대', '##함', '##의', '끝', '"', '(', 'End']   /   거대함의 끝
['균', '##질', '##성', '##과', ',']   /   균질성
['12', '##억', '광', '##년', '##임을']   /   12억 광년
['"', '거', '##대', '##함', '##의', '끝', '"', '(', 'End']   /   거대함의 끝
['260', '/', 'h', 'M', '##p', '##c', '##일']   /   260/h Mpc
['암', '##흑', '##물', '##질', '##이']   /   암흑물질
['바', '##리', '##온', '물', '##질', '##

['상', '##호', '##협', '##조의']   /   상호협조
['순', '##방', '##했다', '.', '(', '1969', '##년', '2월', ')']   /   1969년 2월
['2016년']   /   2016년
['1888', '##년', '##의']   /   1888년
['선', '##거', '##인', '##단', '수', '##에서']   /   선거인단 수
['5', '##번']   /   5번
['286', '##만', '표', ',']   /   286만 표
['1888', '##년', '##의']   /   1888년
['약', '##성', '##서', '##에는']   /   약성서
['발', '##현', '##이', '##라고']   /   발현
['하', '##느', '##님', '##의', '발', '##현', '##이', '##라고']   /   하느님의 발현
['하', '##느', '##님', '##의', '어', '##좌']   /   하느님의 어좌
['어', '##좌']   /   어좌
['약', '##성', '##서', '##에는']   /   약성서
['후', '##기', '유', '##다', '##이', '##즘', '##의']   /   후기 유다이즘
['도', '##구', '##로서']   /   도구
['후', '##기', '유', '##다', '##이', '##즘', '##의']   /   후기 유다이즘
['그리스', '##도는']   /   그리스도
['하', '##느', '##님', '##의', '도', '##구', '##로서']   /   하느님의 도구
['후', '##기', '유', '##다', '##이', '##즘', '##의']   /   후기 유다이즘
['미', '##카', '##엘', '(', '유', '##다']   /   미카엘(유다
['디', '##오', '##니', '##시', '##오', '(', 'Dion', '##ys', '##ius']   /   디오니시오
['신'

['샤', '##이', '##크', '미', '##스', '##킨', '##에서는']   /   샤이크 미스킨
['"', '저', '##항', '##의', '날', '"', '로']   /   저항의 날
['올', '##리', '##브', '가지', '##와']   /   올리브 가지
['올', '##리', '##브', '가지', '##와']   /   올리브 가지
['알', '##미', '##단']   /   알미단
['홈', '##스', '##에서는']   /   홈스
['바', '##니', '##야', '##스', '##에서는']   /   바니야스
['알', '##레', '##포', '##에서는']   /   알레포
['홈', '##스', '##에서는']   /   홈스
['알', '##레', '##포', '##에서는']   /   알레포
['아', '##네', '##르', '##스', '포', '##그', '라', '##스', '##무', '##센', '##은']   /   아네르스 포그 라스무센
['1', ',', '000', '##명을']   /   1,000명
['11', '##명이']   /   11명
['아', '##네', '##르', '##스', '포', '##그', '라', '##스', '##무', '##센', '##은']   /   아네르스 포그 라스무센
['함', '##자', '알', '##하', '##티', '##브']   /   함자 알하티브
['5월', '30일', ',']   /   5월 30일
['바', '##트', '##당', '##은']   /   바트당
['13', '##살']   /   13살
['전', '##국', '##민', '##대', '##화', '##위원회', '##를']   /   전국민대화위원회
["'", '잔', '##혹', '##한', '점', '##령', '##군', "'", '이라고']   /   잔혹한 점령군
['사', '##우', '##디', '##아', '##라', '##비아', '##는']  

['서', '##양', '##화를']   /   서양화
['반', '##올', '##림', '》', '의']   /   반올림
['2004년']   /   2004년
['2003년']   /   2003년
['2004년']   /   2004년
['2003년']   /   2003년
['2004년', '8월', '15일']   /   2004년 8월 15일
['《', '우', '##리에', '##게', '내', '##일', '##은', '없다', '》', '의']   /   우리에게 내일은 없다
['2006년']   /   2006년
['《', '우', '##리에', '##게', '내', '##일', '##은', '없다', '》', '의']   /   우리에게 내일은 없다
['2006년']   /   2006년
['《', '우', '##리에', '##게', '내', '##일', '##은', '없다', '》', '의']   /   우리에게 내일은 없다
['노', '##동', '##석']   /   노동석
['《', '좋', '##지', '아', '##니', '##한', '##가', '》', '에']   /   좋지 아니한가
['평', '##택', '피', '##어', '##선', '영화', '##제', '##에서']   /   평택 피어선 영화제
['2007년', '##에']   /   2007년
['제', '##3', '##회']   /   제3회
['정', '##윤', '##철']   /   정윤철
['[UNK]', '심', '##용', '##태', '[UNK]']   /   심용태
['《', '최', '##강', '##칠', '##우', '》', '에']   /   최강칠우
['남자', '##신', '##인', '##연', '##기', '##상을']   /   남자신인연기상
['《', '서', '##양', '##골', '##동', '##양', '##과', '##자', '##점', '앤', '##티', '##크', '》', '가']   /   서양골동양과자점

['<', '거', '##울', '##에', '대한', '명', '##상', '>', '을']   /   거울에 대한 명상
['《', '검', '##은', '꽃', '》', '으로']   /   《검은 꽃》
['<', '살', '##인', '##자의', '기', '##억', '##법', '>']   /   <살인자의 기억법>
['작', '##가', '##상을']   /   작가상
['《', '아', '##랑', '##은', '왜', '》', '를']   /   아랑은 왜
['《', '검', '##은', '꽃', '》', '을']   /   검은 꽃
['《', '검', '##은', '꽃', '》', '을']   /   검은 꽃
['장', '##편', '《', '나는', '나', '##를', '파', '##괴', '##할', '권', '##리가', '있다', '》', '로']   /   나는 나를 파괴할 권리가 있다
['남', '##진', '##우', '##는']   /   남진우
['2008년']   /   2008년
['남', '##진', '##우', '##는']   /   남진우
['허', '##무', '##주의', '##적']   /   허무주의적
['《', '빛', '##의', '##제', '##국', '》', ',']   /   빛의제국
['《', '아', '##랑', '##은', '왜', '》', '까', '##지']   /   아랑은 왜
['선', '##보', '##였다', '.', '《', '퀴', '##즈', '##쇼', '》', '는']   /   퀴즈쇼
['<', '나는', '나', '##를', '파', '##괴', '##할', '권', '##리가', '있다', '>', '는']   /   나는 나를 파괴할 권리가 있다
['베', '##스트', '##극', '##장에서']   /   베스트극장
['뮤', '##지', '##컬', '##로']   /   뮤지컬
['《', '나는', '나', '##를', '파', '##괴', '##할', '권',

['1946', '##년', '11월', '24일']   /   1946년 11월 24일
['엘', '##리', '##너', '루', '##이지', '코', '##웰', '##에게', '##서']   /   엘리너 루이지 코웰
['엘', '##리', '##너', '루', '##이지', '코', '##웰', '##에게', '##서']   /   엘리너 루이지 코웰
["'", '로', '##이드', '마', '##셜', "'", '이라고']   /   로이드 마셜
['사', '##뮤', '##엘', '코', '##웰', '(', 'Samuel']   /   사뮤엘 코웰
['사', '##뮤', '##엘', '코', '##웰', '##과', '엘', '##리', '##너', '코', '##웰', '##의']   /   사뮤엘 코웰과 엘리너 코웰
['필', '##라', '##델', '##피', '##아']   /   필라델피아
['스', '##티', '##븐', '미', '##쇼', '##와', '휴', '애', '##인', '##스', '##워', '##스', '##에게']   /   스티븐 미쇼와 휴 애인스워스
['3', '##살', '##때', '##까지']   /   3살
['1969', '##년', '##전', '##까지']   /   1969년
['사', '##촌', '##이']   /   사촌
['1969', '##년', '##전', '##까지']   /   1969년
['줄', '##리아', '##가']   /   줄리아
['우', '##울', '##증', '##을']   /   우울증
['사', '##뮤', '##엘', '코', '##웰', '##은']   /   사뮤엘 코웰
['사', '##뮤', '##엘', '코', '##웰', '##은']   /   사뮤엘 코웰
['줄', '##리아', '##가']   /   줄리아
['외', '##할', '##머', '##니', '##가']   /   외할머니
['사', '##뮤', '##엘', '코', '##웰

In [130]:
# 정답 단어 => 정답 토큰 매칭 전처리 과정 개선 필요할듯

## Dataset 생성

### class 선언

In [109]:
class KorquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

val_dataset = KorquadDataset(val_encodings)
# train_dataset = KorquadDataset(train_encodings)

### data 확인

In [110]:
# dataset의 타입,크기 확인

print(type(val_dataset))
print(len(val_dataset))
# print(len(train_dataset))

<class '__main__.KorquadDataset'>
5774


## pre_trained model을 활용하여 train

### pre_trained model 세팅

In [111]:
from transformers import BertModel
model = BertModel.from_pretrained('bert-base-multilingual-cased')

### model train

In [112]:
model.to(device)
model.train()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

## dataset을 dataloader로 불러오기

### dataloader 세팅

In [113]:
from torch.utils.data import DataLoader
# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

### sample data 확인

In [114]:
# print(next(iter(train_loader)))

In [115]:
print(type(val_loader), '\n')
print(len(val_loader))
print(len(next(iter(val_loader))['input_ids'][0]), '\n')
print(next(iter(val_loader)))

<class 'torch.utils.data.dataloader.DataLoader'> 

5774
512 

{'input_ids': tensor([[   101,  76485,  17520,  37912,   9565,  10459,  12092,   9027,  36553,
           9929,  28143,   9485,  31166,   9689,  12092,  11102,   9980,  10459,
            113,   9929,  28143,  25549,  19855, 101322,  60469,  68773,  10530,
          20595,  11102,  33768,  88350,  19855,  30134,    114,  29805,   9706,
          16758,  15891,  76036,  13628,    119,  76485,  15361,  46026,  48253,
          12508,  42337, 118625,  99118,  40311,   8896,  34951,  58904,   9644,
          22200,  40958,  10459,   9405,  16617,  17196,  43962,  30858,  35963,
           9323,  14646, 118965,  27303,    119,  18589,   9960,  17253,  40636,
           9926,  37114,  70122,  68767,   9644,  15891,  31720,  10622,   9069,
          37824,  11261,   9901, 118634,  13374,  93222,  30005,  34951,  33768,
          19855,  30134,   9980,  10459,  11287,   9765,  11287,  13628,    119,
           8885,  99118,  10892, 

## optimizer 불러오기

In [116]:
optim = AdamW(model.parameters(), lr=5e-5)

## Linear

In [117]:
linear = torch.nn.Linear(768, 2)
linear.to(device)

Linear(in_features=768, out_features=2, bias=True)

## train (val_dataset & val_loader)

In [131]:
from tqdm import tqdm
val_tqdm = tqdm(val_loader, total=len(val_loader))

total_loss = 0

for i, batch in enumerate(val_tqdm):
    input_ids = batch['input_ids'].to(device, dtype=torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
    start_positions = batch['start_positions'].to(device, dtype=torch.long)
    end_positions = batch['end_positions'].to(device, dtype=torch.long)

    if start_positions > 384:
        continue
    
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    logits = outputs[0]
    logits_after_linear = linear(logits)
    start_logits, end_logits = logits_after_linear.split(1, dim=-1)
    start_logits = start_logits.squeeze(-1)
    end_logits = end_logits.squeeze(-1)
    ignored_index = start_logits.size(1)

    loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    loss = (start_loss + end_loss) / 2
    total_loss += loss.item()
    
#   loss = loss_fn(start_logits, end_logits, start_positions, end_positions)
#     total_loss += loss.item()

    if i % 100 == 0:
        print(i,'/', len(val_loader),'loss : ', loss)
    
    optim.zero_grad()
    loss.backward()
    optim.step()

avg_train_loss = total_loss / len(val_loader)
print("Average training loss: {0:.3f}".format(avg_train_loss))

  0%|          | 1/5774 [00:00<10:56,  8.79it/s]

0 / 5774 loss :  tensor(2.7842, device='cuda:0', grad_fn=<DivBackward0>)


  2%|▏         | 102/5774 [00:10<09:54,  9.53it/s]

100 / 5774 loss :  tensor(4.6646, device='cuda:0', grad_fn=<DivBackward0>)


  3%|▎         | 202/5774 [00:20<09:52,  9.41it/s]

...

5500 / 5774 loss :  tensor(3.3382, device='cuda:0', grad_fn=<DivBackward0>)


 99%|█████████▉| 5702/5774 [09:47<00:07,  9.67it/s]

5700 / 5774 loss :  tensor(4.9635, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 5774/5774 [09:54<00:00,  9.71it/s]

Average training loss: 3.675
